In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics as sk
from sklearn.metrics import confusion_matrix
import networkx as nx
from torch_geometric.typing import SparseTensor
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import torch.nn.functional as F
import torch

In [ ]:
TRAIN_IMAGES_PATH = './data/images/train'
TEST_IMAGES_PATH = './data/images/test'
VAL_IMAGES_PATH = './data/images/val'

TRAIN_NPZ_FILE = './data/npz/train_images.npz'
TEST_NPZ_FILE = './data/npz/test_images.npz'
VAL_NPZ_FILE = './data/npz/val_images.npz'

NUM_FEATURES = 224 * 224 * 3

In [6]:
def load_npz_as_tensors(file_path):
    """
    Load the .npz files as tensors

    Args:
        file_path (string): To get the .npz file and load it as a tensor
    """
    data = np.load(file_path, allow_pickle=True)

    images = data['images']
    labels = data['labels']

    images_tensor = torch.tensor(images, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    return images_tensor, labels_tensor

In [ ]:
def process_data(sparsity):
    """
    Process and save the data in both sparse and dense formats.

    Args:
        sparsity (bool): Whether to process the dataset as sparse or dense
    """
    
    # Load training data
    train_images, train_labels = load_npz_as_tensors(TRAIN_NPZ_FILE)
    train_data = train_images.reshape(train_images.shape[0], NUM_FEATURES)
    num_train = train_images.shape[0]

    # Load validation data
    val_images, val_labels = load_npz_as_tensors(VAL_NPZ_FILE)
    val_data = val_images.reshape(val_images.shape[0], NUM_FEATURES)
    num_val = val_images.shape[0]

    # Load test data
    test_images, test_labels = load_npz_as_tensors(TEST_NPZ_FILE)
    test_data = test_images.reshape(test_images.shape[0], NUM_FEATURES)
    num_test = test_images.shape[0]

    # Concatenate train, validation, and test data
    num_data = num_train + num_val + num_test
    data_feat = np.concatenate((train_data, val_data, test_data), axis=0)
    data_label = np.concatenate((train_labels, val_labels, test_labels), axis=0).reshape(-1)

    # Construct and scale adjacency matrix
    adj_matrix = sk.pairwise.cosine_similarity(data_feat, data_feat)
    adj_matrix = (adj_matrix - adj_matrix.min())/(adj_matrix.max()-adj_matrix.min())

    # Apply sparsity thresholds
    threshold = 0.977 if sparsity else 0.970

    adj_matrix = adj_matrix > threshold

    # Generate masks
    train_mask = np.zeros(num_data, dtype=bool)
    train_mask[:num_train] = True
    val_mask = np.zeros(num_data, dtype=bool)
    val_mask[num_train:num_train + num_val] = True
    test_mask = np.zeros(num_data, dtype=bool)
    test_mask[num_train + num_val:] = True

    # Save masks, features, labels, and edge index
    suffix = 'sparse' if sparsity else 'dense'
    base_path = f"./data/npz/{suffix}"

    np.savez_compressed(f"{base_path}/train_mask.npz", train_mask=train_mask)
    np.savez_compressed(f"{base_path}/val_mask.npz", val_mask=val_mask)
    np.savez_compressed(f"{base_path}/test_mask.npz", test_mask=test_mask)
    np.savez_compressed(f"{base_path}/data_feat.npz", data_feat=data_feat)
    np.savez_compressed(f"{base_path}/data_label.npz", data_label=data_label)

    # Generate and save edge index
    edge_index = np.array([[i, j] for i in range(num_data) for j in range(num_data) if i != j and adj_matrix[i, j]])
    np.savez_compressed(f"{base_path}/edge_index.npz", edge_index=edge_index)

    print(f"View-({'Sparse' if sparsity else 'Dense'}) generated!")